In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<h3>Import the data</h3>

In [ ]:
BNP_diff = pd.read_excel('/kaggle/input/nek-310-lecture-4/BNP_diff.xlsx', sheet_name='Ekonomifakta')

Unemp = pd.read_excel('/kaggle/input/nek-310-lecture-4/Unemp.xlsx', sheet_name='Ekonomifakta')

<h3>Check what is in the datasets</h3>

In [ ]:
Unemp.head()

In [ ]:
Unemp.columns

In [ ]:
Unemp.dtypes

<h3>We don't want month as an object. Let's try to make it a date</h3>

In [ ]:
Unemp['Month_2'] = pd.to_datetime(['Month'])

In [ ]:
Unemp['Month1']=Unemp['Month'].astype('string')

In [ ]:
Unemp.dtypes

In [ ]:
Unemp.head()

<h3>Now month is a string, let's see if that helps...</h3>

In [ ]:
import datetime as dt



Unemp['Date'] = pd.to_datetime(Unemp['Month1'].apply(lambda x: dt.datetime.strptime(x, '%b-%y')))

<h3>Pandas don't seem to speak Swedish...Let's see if we can recode/translate...</h3>

In [ ]:
def month_recode(month):
    if month == "maj":
        return 'may'
    if month == 'okt':
        return 'oct'
    else:
        return month

In [ ]:
month_recode('maj')

In [ ]:
Unemp['Month_2']=Unemp['Month'].str.slice(0,3).apply(month_recode)

In [ ]:
Unemp['Month_4']=Unemp['Month_2'].str.cat(Unemp['Month1'].str[-2:],sep='-').astype('string')

In [ ]:
Unemp.head(20)

In [ ]:
Unemp.dtypes

In [ ]:
import datetime as dt



Unemp['Date']=pd.to_datetime(Unemp['Month_4'], format='%b-%y')

In [ ]:
Unemp.dropna(axis=0, inplace=True)

In [ ]:
Unemp['Date']=pd.to_datetime(Unemp['Month_4'], format='%b-%y') + pd.offsets.MonthEnd()

In [ ]:
Unemp.head()

<h3>Now let's try and make a time series plot</h3>

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

plt.plot(Unemp['Date'],Unemp['Procent'])

<h3>Can we make a time series plot for BNP growth</h3>

In [ ]:
BNP_diff.head()

In [ ]:
BNP_diff.dropna(axis=0, inplace=True)

<h3>We want the quarters 'kv1', 'kv2', 'kv3' and 'kv4' as dates, </h3>

In [ ]:
def quarter_recode(quarter):
    if quarter == ", kv 1":
        return '03-31'
    if quarter == ', kv 2':
        return '06-30'
    if quarter == ', kv 3':
        return '09-30'
    if quarter == ', kv 4':
        return '12-31'
    else:
        return 'Error' 

In [ ]:
quarter_recode(', kv 1')

In [ ]:
BNP_diff['Year']=BNP_diff['Quarter'].str.slice(0,4)

In [ ]:
BNP_diff['Date']=pd.to_datetime(BNP_diff['Year'].str.cat(BNP_diff['Quarter'].str[-6:].apply(quarter_recode),sep='-').astype('string'),format='%Y-%m-%d')

In [ ]:
BNP_diff.head()

<h3>Drop columns we do not need and rename</h3>

In [ ]:
BNP_diff_2=BNP_diff.rename(columns={'Procent, jämfört med föregående kvartal': 'Percent_inc'})

In [ ]:
BNP_diff_2.drop(columns=['Quarter','Procent, jämfört med samma kvartal föregående år','Date_pre'])

<h3>Make a time series plot</h3>

In [ ]:
plt.plot(BNP_diff_2['Date'],BNP_diff_2['Percent_inc'])